In [ ]:
!pip install "gymnasium[mujoco]"
!pip install mpi4py

In [ ]:
import os
import subprocess

In [ ]:
env_file = 'model_downloader.py'
env_url = "https://raw.githubusercontent.com/alibaniasad1999/master-thesis/main/Code/Python/utils/model_downloader.py"

if not os.path.isfile(env_file):
    print(f"Downloading {env_file} ...")
    subprocess.run(
        ['wget', '-q', '-O', env_file, env_url],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    print(f"{env_file} downloaded.")
else:
    print(f"{env_file} already exists.")

from model_downloader import download_everything

trajectory = download_everything('sac, TBP')

In [ ]:
import torch
import SAC
from SAC import SAC
from TBP import ThreeBodyEnv
from utils.run_utils import setup_logger_kwargs
from SAC import colorize

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(colorize(f"Device is {device}! 🥰😎", 'green', bold=True))
# Define variables directly
env = ThreeBodyEnv(trajectory)
hid = 64
l = 2
gamma = 0.99
seed = 0
epochs = 50
exp_name = 'ddpg'
steps_per_epoch_=30000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ac_kwargs = dict(hidden_sizes=[hid]*l)
states_array = []
# logger_kwargs = setup_logger_kwargs(exp_name, seed)
reward_array=[]
# ddpg = DDPG(lambda : ThreeBodyEnv(trajectory, error_range=0.01, final_range=0.001), MLPActorCritic, ac_kwargs,seed_=seed, steps_per_epoch=steps_per_epoch_, epochs_=epochs, logger_kwargs_=logger_kwargs, device=device, batch_size=1024, start_steps=5000, update_after=1000, update_every=2000)


sac_rl = SAC(lambda : ThreeBodyEnv(trajectory, error_range=0.01, final_range=0.001), epochs=100, device_class=device, ac_kwargs_class=ac_kwargs, steps_per_epoch=steps_per_epoch_, batch_size=1024, start_steps=5000, update_after=1000, update_every=2000)

In [ ]:
sac_rl.train(epochs=10)

In [ ]:
sac_rl.test(fun_mode=True, deterministic=True, save_data=False)

In [ ]:
sac_rl.save()

In [ ]:
sac_rl.load()

In [ ]:
try:
    # noinspection PyUnresolvedReferences
    import google.colab
    # download created models
    if os.path.isfile("sac_TBP_results.tar.gz"):
        os.remove("sac_TBP_results.tar.gz")
    !tar cvfz sac_TBP_results.tar.gz model/
    # noinspection PyUnresolvedReferences
    from google.colab import files
    files.download('sac_TBP_results.tar.gz')
    print(colorize("Done! 🥰😎", 'blue', bold=True))
except Exception as e:
    print(e, end=' ----> ')
    print(colorize("Something Wrong 🧐😩, or maybe you used your Mac 😱🥲", 'red', bold=True))